In [1]:
# !pip install datasets
# !pip install torchmetrics
# !pip install pytz
# !pip install persiantools
# !pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Mount Google Drive

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# %cd /content/drive/MyDrive

/content/drive/MyDrive


# Configuration

In [4]:
config = {}
config['root_path'] = '/home/user01/'
config['series_ID'] = 103
config['series_desc'] = '''
targeted backdoor attack labels changed - LR influence attack manually implemented (without the help of jax)
'''
config['log_path'] = config['root_path']+'experiments/MNIST/LR/'
config['log'] = {}

In [5]:
config['poisoning_rate'] = 0.2
# config['num_clean_examples'] = 200
config['learning_rate'] = 0.01
config['batch_size'] = 32
config['num_epochs'] = 2
config['attack_step_size'] = 0.1
config['num_iters'] = 1000

In [6]:
config['log']['model'] = 'LR'
config['log']['dataset'] = 'MNIST (0-1)'
config['log']['task'] = 'binary classification'
config['log']['pytorch_seed'] = 50
config['log']['numpy_seed'] = 50
config['log']['projection'] = 'R^400'
config['log']['attack'] = 'influence'
config['log']['method'] = 'modify'

In [7]:
config['log_path'] += (str(config['series_ID']) + '-' + config['log']['attack'] + '-' + config['log']['dataset'] + '-' + str(int(config['poisoning_rate']*100)) + '-' + config['log']['method'] + '.json').lower().replace(' ', '')

In [8]:
config['log_path']

'/content/drive/MyDrive/experiments/MNIST/LR/103-influence-mnist(0-1)-20-modify.json'

In [9]:
### WE NEED THIS TO IMPORT THE NECESSARY LIBRARIES ###
import sys
sys.path.append(config['root_path'])

In [10]:
import datascience, poisoning, report
from datascience.data import CIFAR10, MNIST, IMDB, BOSTON
from datascience.general import train_dev_test_split, join_np_arrays, describe_dataset, randomized_round
from poisoning.process import attacker, defender, SVM_influence_attacker, LR_influence_attacker
from poisoning.eval import attack_success_rate, benign_accuracy, test_accuracy
from report.log import JSONLogger, TextLogger, tehran_datetime
from temporary.functions import _reload
import numpy as np
import torch
from torchmetrics import HingeLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd.functional import hessian, jacobian
from torch.autograd import grad
from torch.nn.utils import _stateless
from datetime import datetime
import pytz
from persiantools.jdatetime import JalaliDate
import warnings
warnings.filterwarnings("ignore")

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [11]:
_reload(poisoning.process)
_reload(poisoning.eval)
_reload(datascience.data)
_reload(datascience.general)
_reload(report.log)

<module 'report.log' from '/content/drive/MyDrive/report/log.py'>

In [12]:
torch.manual_seed(config['log']['pytorch_seed'])

# Loading a Dataset

In [13]:
dataset = MNIST()
dataset.select_labels([0,1], phase='train')
dataset.select_labels([0,1], phase='test')
dataset.resize(20,20)
dataset.flatten()
dataset.rescale()
dataset.random_sample(500, 'train')
dataset.random_sample(100, 'test')

In [14]:
# dataset.change_labels({0:-1})

In [15]:
# dataset.separate_examples(config['num_clean_examples'], desired_seed = config['log']['numpy_seed'])

# Attack

In [16]:
att = LR_influence_attacker(dataset.x_train, dataset.y_train, dataset.x_test, dataset.y_test, config['poisoning_rate'], config['attack_step_size'], config['log']['method'])

In [ ]:
%%time
att.attack(num_iters=config['num_iters'])

In [ ]:
y_p = np.array([1 if _y==1 else 0 for _y in y_p])
# dataset.revert_labels()

In [ ]:
y_train_all = np.concatenate((y_p, dataset.y_train), axis=0)

In [ ]:
x_train_all = np.concatenate((x_p, dataset.x_train), axis=0)

In [ ]:
config['data-train'] = describe_dataset(x_train_all, y_train_all, 'training dataset')

In [ ]:
config['data-test'] = describe_dataset(dataset.x_test, dataset.y_test, 'testing dataset')

# Prepare the Data

In [ ]:
class MyVectorDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = np.array(labels).reshape(-1, 1)
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self, idx):
        return torch.Tensor(self.features[idx]).to(device), torch.Tensor(self.labels[idx]).to(device)

In [ ]:
train_dataset = MyVectorDataset(x_train_all, y_train_all)
# train_dataset = MyVectorDataset(dataset.x_train, dataset.y_train)
test_dataset = MyVectorDataset(dataset.x_test, dataset.y_test)
# clean_dataset = MyVectorDataset(dataset.x_train_clean, dataset.y_train_clean)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)
# clean_dataloader = DataLoader(clean_dataset, batch_size=config['batch_size'], shuffle=True)

# Train a Model

In [ ]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

In [ ]:
model = LogisticRegression(400,1)
model = model.to(device)
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], weight_decay=1e-5)

In [ ]:
def output_to_label(out):
    if out >= 0.5:
      return 1
    else:
      return 0

In [ ]:
def make_prediction(model, x_arr):
  outs = list(model(torch.Tensor(x_arr)).squeeze().detach().numpy())
  labels = [output_to_label(out) for out in outs]
  return np.array(labels)

In [ ]:
# class SVM(torch.nn.Module):
#     def __init__(self):
#         super(SVM, self).__init__() 
#         self.linear = torch.nn.Linear(in_features=784, out_features=1, bias=True)
#     def forward(self, x):
#         output = self.linear(x)
#         return output

In [ ]:
# model = SVM()
# loss_fn = HingeLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], weight_decay=1e-5)

In [ ]:
# def output_to_label(out):
#     if out >= 0:
#       return 1
#     else:
#       return 0

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch_num):
    num_points = len(dataloader.dataset)
    for batch, (features, labels) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(features)
        loss = loss_fn(pred, labels)
        
        # Backpropagation
        optimizer.zero_grad() # sets gradients of all model parameters to zero
        loss.backward() # calculate the gradients again
        optimizer.step() # w = w - learning_rate * grad(loss)_with_respect_to_w

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(features)
            print(f"\r Epoch {epoch_num} - loss: {loss:>7f}  [{current:>5d}/{num_points:>5d}]", end=" ")

In [ ]:
def test_loop(dataloader, model, loss_fn, epoch_num, name):
    num_points = len(dataloader.dataset)
    sum_test_loss, correct = 0, 0

    with torch.no_grad():
        for batch, (features, labels) in enumerate(dataloader):
            pred = model(features)
            sum_test_loss += loss_fn(pred, labels).item() # add the current loss to the sum of the losses
            # convert the outputs of the model on the current batch to a numpy array
            pred_lst = list(pred.cpu().numpy().squeeze())
            pred_lst = [output_to_label(item) for item in pred_lst]
            # convert the original labels corresponding to the current batch to a numpy array
            output_lst = list(labels.cpu().numpy().squeeze()) 
            # determine the points for which the model is correctly predicting the label (add a 1 for each)
            match_lst = [1 if p==o else 0 for (p, o) in zip(pred_lst, output_lst)] 
            # count how many points are labeled correctly in this batch and add the number to the overall count of the correct labeled points
            correct += sum(match_lst) 
            
    sum_test_loss /= num_points
    correct /= num_points
    config['log']['accuracy_'+name] = (100*correct)
    config['log']['loss_'+name] = sum_test_loss
    print(f"\r Epoch {epoch_num} - {name} Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {sum_test_loss:>8f}", end=" ")

In [ ]:
for epoch_num in range(1, config['num_epochs']+1):
    train_loop(train_dataloader, model, loss_fn, optimizer, epoch_num)

 Epoch 2 - loss: 0.550278  [    0/ 1800] 

In [ ]:
test_loop(train_dataloader, model, loss_fn, config['num_epochs'], 'Train')

 Epoch 2 - Train Error: Accuracy: 78.1%, Avg loss: 0.014725 

In [ ]:
test_loop(test_dataloader, model, loss_fn, config['num_epochs'], 'Test')

 Epoch 2 - Test Error: Accuracy: 100.0%, Avg loss: 0.010497 

In [ ]:
config['datetime'] = tehran_datetime()

In [ ]:
config

{'log': {'model': 'LR',
  'dataset': 'MNIST',
  'task': 'binary classification (0-1) - Influe Attack',
  'pytorch_seed': 50,
  'numpy_seed': 50,
  'projection': 'R^400',
  'accuracy_Train': 78.11111111111111,
  'loss_Train': 0.014724785387516022,
  'accuracy_Test': 100.0,
  'loss_Test': 0.010496590584516526},
 'log_path': '/content/drive/MyDrive/experiments/MNIST/LR/influence-01-80-no-assumed-defense.json',
 'poisoning_rate': 0.8,
 'learning_rate': 0.01,
 'batch_size': 32,
 'num_epochs': 2,
 'attack_step_size': 0.1,
 'num_iters': 3,
 'data-train': {'name': 'training dataset',
  'num_samples': 1800,
  'num_features': (400,),
  'class_count': {0: 884, 1: 916}},
 'data-test': {'name': 'testing dataset',
  'num_samples': 100,
  'num_features': (400,),
  'class_count': {0: 50, 1: 50}},
 'datetime': '1401-07-14 18:44'}

In [ ]:
logger = JSONLogger(config['log_path'], config)
logger.log()

In [ ]:
randomized_round(-1.3)

array(-2.)